In [1]:
%cd /data/jupyter/root/ECNU/RecommendeSystems/HomeWork1/
%ls

/data/jupyter/root/ECNU/RecommendeSystems/HomeWork1
data/  wk1 user cf.ipynb


In [97]:
# %cat data/ml-1m/movies.dat

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 数据预处理

## 转换ratings数据

In [98]:
f = open('data/ratings.csv', "w+")
f.truncate()
# 数据太大内存存不下，分块处理
chunks = pd.read_table("data/ml-1m/ratings.dat", sep="::", engine='python', chunksize = 10000, names=[
    "UserID", 
    "MovieID",
    "Rating", 
    "Timestamp"]
)
for chunk in chunks:
#     print(chunk)
    chunk.to_csv('data/ratings.csv', mode='a+', header=False, index=False)

In [4]:
ratings = pd.read_csv("data/ratings.csv", names=[
    "UserID", 
    "MovieID",
    "Rating", 
    "Timestamp"])

In [12]:
ratings

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


## 转换movies数据

In [13]:
movies = pd.read_table("data/ml-1m/movies.dat", sep="::", engine='python', names=[
    "MovieID",
    "Title",
    "Genres"]
)

In [14]:
movies

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


## 连接2个表

In [15]:
data = pd.merge(movies, ratings, on = 'MovieID')
data[['UserID','Rating','MovieID','Title']].sort_values('UserID').to_csv('data/data.csv',index=False)
data

,MovieID,Title,Genres,UserID,Rating,Timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474
...,...,...,...,...,...,...
1000204,3952,"Contender, The (2000)",Drama|Thriller,5812,4,992072099
1000205,3952,"Contender, The (2000)",Drama|Thriller,5831,3,986223125
1000206,3952,"Contender, The (2000)",Drama|Thriller,5837,4,1011902656
1000207,3952,"Contender, The (2000)",Drama|Thriller,5927,1,979852537


In [16]:
data = pd.read_table('data/data.csv', sep=",")
data

,UserID,Rating,MovieID,Title
0,1,5,1,Toy Story (1995)
1,1,4,1207,To Kill a Mockingbird (1962)
2,1,4,1246,Dead Poets Society (1989)
3,1,4,2762,"Sixth Sense, The (1999)"
4,1,4,531,"Secret Garden, The (1993)"
...,...,...,...,...
1000204,6040,4,3108,"Fisher King, The (1991)"
1000205,6040,4,541,Blade Runner (1982)
1000206,6040,4,3505,No Way Out (1987)
1000207,6040,5,3334,Key Largo (1948)


# 数据转化

In [3]:
import pandas as pd
import numpy as np
import warnings
import random
warnings.filterwarnings('ignore')

def process_bar(percent, start_str='', end_str='100%', total_length=0):
    bar = ''.join(["\033[31m%s\033[0m"%'   '] * int(percent * total_length)) + ''
    bar = '\r' + start_str + bar.ljust(total_length) + ' {:0>4.1f}%|'.format(percent*100) + end_str
    print(bar, end='', flush=True)


def split_data(data, m, k, seed):
    train = pd.DataFrame()
    test = pd.DataFrame()

    random.seed(seed)
    for index, row in data.iterrows():
        process_bar(index / len(data))
        train_row = row.copy()
        test_row = row.copy()

        for index_, item in row.iteritems():
            if not item:
                continue

            if not random.randint(0, m/k):
                test_row[index_] = 0
            else:
                train_row[index_] = 0

        train = train.append(train_row)
        test = test.append(test_row)
    return train, test


def covert_user_item_mat(data):
    data_mat = pd.DataFrame()
    length = len(data)
    for index, row in data.iterrows():

        u_id = row["UserID"]
        m_id = row["MovieID"]
        rating = row["Rating"]

        movie_ids = data_mat.columns
        if u_id not in data_mat.index:
            new_user = pd.Series(np.zeros(len(movie_ids)), index=movie_ids, name=u_id)
            data_mat = data_mat.append(new_user)

        user_ids = data_mat.index
        if m_id not in data_mat.columns:
            new_movie = pd.Series(np.zeros(len(user_ids)), index=user_ids, name=m_id)
            new_movie[u_id] = rating
            data_mat[m_id] = new_movie
        else:
            data_mat.loc[u_id][m_id] = rating

    data_mat = data_mat.reindex(sorted(data_mat.columns), axis=1)
    return data_mat


data = pd.read_table('data/data.csv', sep=",")[:1000]
u_i_mat = covert_user_item_mat(data)
# u_i_mat.to_csv('data/data_mat_train.csv', header=True, index=False)

# u_i_mat = pd.read_csv('data/data_mat.csv')

train_mat, test_mat = split_data(u_i_mat, 8, 2, 1)

# train_mat.to_csv('data/data_mat_train.csv', header=True, index=False)
# test_mat.to_csv('data/data_mat_test.csv', header=True, index=False)
# train_mat = pd.read_csv('data/data_mat_train.csv').loc[0].sum()
# test_mat = pd.read_csv('data/data_mat_test.csv').loc[1].sum()

# r1 = train_mat.loc[2]
# r2 = test_mat.loc[2]
# print(len(r1[r1>0]))
# print(len(r2[r2>0]))

 100.0%|100%

In [9]:
train_mat

,1,2,4,6,7,14,16,17,21,24,...,3799,3809,3812,3826,3868,3869,3893,3916,3928,3948
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
test_mat

,1,2,4,6,7,14,16,17,21,24,...,3799,3809,3812,3826,3868,3869,3893,3916,3928,3948
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4.0,0.0,3.0,0.0,0.0,4.0,4.0,4.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0
10,5.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0


# User/Item CF 推荐

In [ ]:
import pandas as pd
import numpy as np
import warnings
import random
from numpy import linalg as la
warnings.filterwarnings('ignore')
data = pd.read_table('data/data.csv', sep=",", index_col="UserID")[:1000]


def process_bar(percent, start_str='', end_str='100%', total_length=0):
    bar = ''.join(["\033[31m%s\033[0m"%'   '] * int(percent * total_length)) + ''
    bar = '\r' + start_str + bar.ljust(total_length) + ' {:0>4.1f}%|'.format(percent*100) + end_str
    print(bar, end='', flush=True)


def eclud_sim(a, b):
    return 1.0/(1.0 + la.norm(a - b))

def pears_sim(a, b):
    if len(a) < 3:
        return 1.0
    return 0.5+0.5*np.corrcoef(a, b, rowvar=False)[0][1]

def cons_sim(a,b):
    num = float(a.dot(b))
    denom = la.norm(a) * la.norm(b)
    return 0.5+0.5*(num/denom)

def jaccard(a, b):
    return 1.0*(a*b).sum()/(a+b-a*b).sum()

class Recommender():
    sim = None
    data = None
    def similarity(self, x, distance):
        y = np.ones((len(x), len(x)))
        for i in range(len(x)):
            process_bar(i / len(x))
            for j in range(len(x)):
                y[i, j] = distance(x[i], x[j])
        return y

    def fit(self, x, distance=jaccard):
        self.data = x
        self.sim = self.similarity(x.values, distance)


    def recommend_item(self, user_row, sim_row):
        recommendations = []
        for index1, item1 in user_row.iteritems():
            for index2, item2 in sim_row.iteritems():
                if item2 !=0 and item1 == 0:
                    recommendations.append((index2, item2))
        recommendations.sort(key=lambda val: val[1], reverse=True)
        return recommendations

    def recommend_user_cf(self, user_id, N):
        a = self.sim[0][1:]
        sim_user_index = np.where(a==np.max(a))[0][0]

        # index = id
        user_row = self.data.loc[user_id]
        sim_row = self.data.loc[sim_user_index]


        return self.recommend_item(user_row, sim_row)[:N]

    def recommend_item_cf(self, user_id, N):
        user_clo = self.data[user_id]
        sim_items = []
        for index, item in user_clo.iteritems():
            if item == 0:
                continue

            a = self.sim[index][1:]
            sim_item_indexs = np.argsort(a)

            for index in sim_item_indexs:
                sim_items.append(self.data.loc[index])

        return sim_items[:N]


    def recall(self, train, test, N=10):
        hit = 0
        totla_recall = 0
        totla_precision = 0
        for index, row in train.iterrows():
            process_bar(index / len(train))

            test_row = test.loc[index]

            if test_row.empty:
                continue

            res = self.recommend_user_cf(index)[:N]


            for res_item in res:
                if res_item[0] in test_row[test_row>0]:
                    hit += 1
            totla_recall += len(test_row[test_row>0])
            totla_precision += N
        return hit / (totla_recall * 1.0), hit / (totla_precision * 1.0)


train_mat = pd.read_csv('data/data_mat_train.csv', sep=",")[:100]
test_mat = pd.read_csv('data/data_mat_test.csv', sep=",")[:100]

r = Recommender()
r.fit(train_mat, cons_sim)
print('\n', r.recommend_user_cf(1,10))
# print('\n', r.recall(train_mat, test_mat))

r.fit(train_mat.T, cons_sim)
print('\n', r.recommend_item_cf(1,10))
# print('\n', r.recall(train_mat, test_mat))


# Matrix Factorization (矩阵分解)

In [5]:
import pandas as pd
import numpy as np
num_user = np.max(ratings["UserID"])
num_movie = np.max(ratings["MovieID"])
print(num_user,num_movie,len(ratings))

6040 3952 1000209


In [ ]:
from keras import Model
import keras.backend as K
from keras.layers import Embedding,Reshape,Input,Dot
K.clear_session()
def Recmand_model(num_user,num_movie,k):
    input_uer = Input(shape=[None,],dtype="int32")
    model_uer = Embedding(num_user+1,k,input_length = 1)(input_uer)
    model_uer = Reshape((k,))(model_uer)
    
    input_movie = Input(shape=[None,],dtype="int32")
    model_movie  = Embedding(num_movie+1,k,input_length = 1)(input_movie)
    model_movie = Reshape((k,))(model_movie)
    
    out = Dot(1)([model_uer,model_movie])
    model = Model(inputs=[input_uer,input_movie], outputs=out)
    model.compile(loss='mse', optimizer='Adam')
    model.summary()
    return model

Using TensorFlow backend.


In [ ]:
model = Recmand_model(num_user,num_movie,100)

In [ ]:
train_user = ratings["UserID"].values
train_movie = ratings["MovieID"].values
train_x = [train_user,train_movie]
train_y = ratings["Rating"].values

In [ ]:
model.fit(train_x,train_y,batch_size = 100,epochs =10)

In [ ]:
model.predict([[1],[2]])